In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/nlp_final

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive/My Drive/nlp_final


In [0]:
import pandas as pd
import numpy as np

In [0]:
def convert_test(df):
    df1 = df.drop('RandomFifthSentenceQuiz2', axis=1)
    df2 = df.drop('RandomFifthSentenceQuiz1',axis=1)
    df1.rename(columns={'RandomFifthSentenceQuiz1':'Quiz'}, inplace=True)
    df2.rename(columns={'RandomFifthSentenceQuiz2':'Quiz'}, inplace=True)
    
    df1["Story"] = df1['InputSentence1'].str.cat(df1['InputSentence2'])
    df1["Story"] = df1['Story'].str.cat(df1['InputSentence3'])
    df1["Story"] = df1['Story'].str.cat(df1['InputSentence4'])
    df1 = df1.drop(columns=['InputSentence1','InputSentence2','InputSentence3','InputSentence4'])
    df1 = df1[['InputStoryid','Story','Quiz','AnswerRightEnding']]
    
    df2["Story"] = df2['InputSentence1'].str.cat(df2['InputSentence2'])
    df2["Story"] = df2['Story'].str.cat(df2['InputSentence3'])
    df2["Story"] = df2['Story'].str.cat(df2['InputSentence4'])
    df2 = df2.drop(columns=['InputSentence1','InputSentence2','InputSentence3','InputSentence4'])
    df2 = df2[['InputStoryid','Story','Quiz','AnswerRightEnding']]
    
    
    return df1,df2

In [0]:
df_val = pd.read_csv("data/ROC-Story-Cloze-Data.csv")

In [0]:
test1,test2 = convert_test(df_val)
test1.to_csv("tmp/test1/test.tsv",sep="\t",index=None)
test2.to_csv("tmp/test2/test.tsv",sep="\t",index=None)
del df_val

In [0]:
ckpt_file = 'model/model11374.ckpt'

In [7]:

!python bin/story_classifier.py --task_name=STORY --do_predict=true --data_dir=tmp/test1 --vocab_file=bin/pre/vocab.txt --bert_config_file=bin/pre/bert_config.json --init_checkpoint=$ckpt_file --max_seq_length=60 --output_dir=tmp/res1





W1205 23:54:02.410326 139897159702400 module_wrapper.py:139] From bin/story_classifier.py:784: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W1205 23:54:02.410547 139897159702400 module_wrapper.py:139] From bin/story_classifier.py:784: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W1205 23:54:02.411006 139897159702400 module_wrapper.py:139] From /gdrive/My Drive/nlp_final/bin/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W1205 23:54:02.776289 139897159702400 module_wrapper.py:139] From bin/story_classifier.py:808: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https

In [8]:

!python bin/story_classifier.py --task_name=STORY --do_predict=true --data_dir=tmp/test2 --vocab_file=bin/pre/vocab.txt --bert_config_file=bin/pre/bert_config.json --init_checkpoint=$ckpt_file --max_seq_length=60 --output_dir=tmp/res2





W1205 23:58:02.237512 140706757760896 module_wrapper.py:139] From bin/story_classifier.py:784: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W1205 23:58:02.237747 140706757760896 module_wrapper.py:139] From bin/story_classifier.py:784: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W1205 23:58:02.238214 140706757760896 module_wrapper.py:139] From /gdrive/My Drive/nlp_final/bin/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W1205 23:58:02.239976 140706757760896 module_wrapper.py:139] From bin/story_classifier.py:808: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https

In [0]:
res1 = pd.read_csv("tmp/res1/test_results.tsv",sep="\t",header=None)
res2 = pd.read_csv("tmp/res2/test_results.tsv",sep="\t",header=None)

In [10]:
df_out = test1[["InputStoryid","AnswerRightEnding"]]
df_out["score1"] = res1[1]
df_out["score2"] = res2[1]
scores = df_out[["score1","score2"]].values
pred = np.argmax(scores,1)+1
df_out["pred"] = pred
acc = np.mean(df_out["pred"].values == df_out["AnswerRightEnding"].values)
print(acc)

0.9857350096711799


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
#save df_out as output
df_out

,InputStoryid,AnswerRightEnding,score1,score2,pred
0,affdc682-1351-453d-9ab8-64c9b69936fc,1,0.634433,0.276924,1
1,bdc7f54a-a194-49a0-8b1e-08e8d05a4168,2,0.419270,0.630753,2
2,e03003a1-f95d-4a6e-acad-e1d7fc3b1068,2,0.300509,0.617618,2
3,ba35049a-13f5-4862-b004-f0b3c6f4f40c,2,0.328546,0.623197,2
4,a1f1395d-5cc9-4691-af60-fc2ebeaf2feb,1,0.626571,0.337056,1
...,...,...,...,...,...
45491,3097bc2d-a822-4708-a9ba-3610c07334e2,2,0.296338,0.459105,2
45492,1d6b0355-aba8-4a02-903e-301a21660896,1,0.629963,0.453712,1
45493,41a03bfe-4e65-4434-afe5-4305033f1982,2,0.293723,0.618781,2
45494,62170547-3529-4137-a91c-5646e14856f7,1,0.562180,0.274086,1


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

In [83]:
df_out = df_out[['score1','score2','pred','AnswerRightEnding']]
X_train, X_test, Y_train, Y_test = train_test_split(df_out[['score1','score2']].values, df_out[['AnswerRightEnding']].values,random_state=42)
logistic_clf = LogisticRegression().fit(X_train,Y_train)
RF_clf = RandomForestClassifier().fit(X_train,Y_train)
SVM_clf = SVC().fit(X_train,Y_train)
NN_clf = MLPClassifier(hidden_layer_sizes=(2)).fit(X_train,Y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/sklearn/util

In [0]:
df_test = pd.DataFrame(X_test)
df_test['real_label'] = Y_test
df_test['naive_pred'] = np.argmax(X_test,1)+1
df_test['logistic_pred'] = logistic_clf.predict(X_test)
df_test['RF_pred'] = RF_clf.predict(X_test)
df_test['SVM_pred'] = SVM_clf.predict(X_test)
df_test['NN_pred'] = NN_clf.predict(X_test)

In [0]:
naive_acc = np.mean(df_test["real_label"].values == df_test["naive_pred"].values)
logistic_acc = np.mean(df_test["real_label"].values == df_test["logistic_pred"].values)
RF_acc = np.mean(df_test["real_label"].values == df_test["RF_pred"].values)
SVM_acc = np.mean(df_test["real_label"].values == df_test["SVM_pred"].values)
NN_acc = np.mean(df_test["real_label"].values == df_test["NN_pred"].values)

In [86]:
[naive_acc,logistic_acc,RF_acc,SVM_acc,NN_acc]

[0.9850536310884473,
 0.984789871637067,
 0.9830314752945314,
 0.9848777914541937,
 0.9848777914541937]

In [15]:
clf.coef_

array([[-20.26415436,  20.25151142]])

In [16]:
clf.intercept_

array([-0.08819332])